In [1]:
__author__ = 'Jeff Starn'
%matplotlib inline
import os
import geopandas as gp
import gdal, osr
import mplleaflet

The purpose of this notebook is to create a shapefile for use in General Model notebooks. The shapefile can be created any way. This notebook provides one convenient way to do it.

Set up pathnames

In [2]:
nhd_dir = 'C:/General_Models_WRR/input_data/NHDPlusV2Data'
nhd_basin_dir = 'NHDPlusNE/NHDPlus01'

The shapefile will be created from the NHDPlus WBD (watershed boundaries) shapefile.

In [3]:
domain_file = os.path.join(nhd_dir, nhd_basin_dir, 'WBDSnapshot', 'WBD', 'WBD_Subwatershed.shp')

Use geopandas to read in the NHDPlus shapefile

In [4]:
domain = gp.read_file(domain_file)

The next variable is used to select the basin(s) of interest. Any basin name that at least partially matches the string will be selected. 

In [5]:
bname = 'Concord'

List the basin with the selected names. Some names may apply to more than one basin.  The state id, name, and HUC10 codes can be used interactively to find the basin to be selected from ones having similar or identical names.

In [6]:
# group = domain.groupby(['HUC_12', 'STATES'])
group = domain.groupby(['HUC_10', 'STATES'])
first = group.HU_10_NAME.first()
first[first.str.contains(bname)]

HUC_10      STATES
0107000502  MA        Concord River
Name: HU_10_NAME, dtype: object

Cut and paste the first 8 digits (HUC8) that are to be selected into the list below.

In [7]:
WS = ['01070005']
#WS = ['01020005','01020004','01020003','01020002','01020001'] # Example of multiple basin numbers input directly
ind = [item in WS for item in domain.HUC_8]
# ind = [item in WS for item in domain.HUC_12]
subdomain = domain[ind]

Open the NED grid from NHDPlus to get the projection. NHDPlus lines are in geographic coordinates rather than a projected coordinate reference system. Extract the projection as WKT, convert it to Proj.4 format, then parse that into a Python dictionary.

In [8]:
src = os.path.join(nhd_dir, nhd_basin_dir, 'NEDSnapshot', 'ned01a', 'elev_cm')
ned = gdal.Open(src)
shapeproj = ned.GetProjection()
srs = osr.SpatialReference()
srs.ImportFromWkt(shapeproj)
prj4 = (srs.ExportToProj4()).split('+')
prj4 = dict([item.split('=') for item in prj4 if len(item.split('=')) == 2])

Use geopandas to extract the GeoSeries Shapely geometry of all the selected basin shapes and project it to the reference coordinate system used by the corrsponding NED grid. Dissolve all the shapes into one Shapely shape. Add the shape to a GeoSeries and then to a GeoDataFrame.  Add an ibound attribute to the dataframe. Save it as a shapefile in the general model directory.

In [9]:
# extract GeoSeries containing polygon shapes
subdomain_shape = subdomain['geometry']
# project the shapes to the project projection
domain_prj = subdomain_shape.to_crs(prj4)
# dissolve the polygons into one shape
domain_diss = gp.GeoSeries(domain_prj.unary_union)
domain_diss = gp.GeoDataFrame(geometry=domain_diss.geometry)
domain_diss['ibound'] = 1
domain_diss.crs = prj4
# write it out to a shapefile
domain_diss.to_file('{}_domain.shp'.format(bname))

Create a Leaflet map to verify you selected the correct basin and that all it's pieces are present.

In [10]:
ax = domain_diss.plot()
mplleaflet.show(fig=ax.figure, crs=domain_diss.crs, path='{}.html'.format(bname))